# Joining and Appending DataFrames in PySpark HW

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('JoiningDataset').getOrCreate()
spark

## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [6]:
import os

path = "data/"

df_list = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".") #separate path from .csv
        df_name = filename_list[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        df_list.append(df_name)
        exec(df_name + ' = df')
        
# QA
print("Full list of dfs:")
print(df_list)

Full list of dfs:
['grade_distributions', 'schedules', 'course_offerings', 'sections', 'subject_memberships', 'teachings', 'instructors', 'courses', 'subjects', 'rooms']


Now check the contents of a few of the dataframses that were read in above.

In [7]:
sections.limit(5).toPandas()

,uuid,course_offering_uuid,section_type,number,room_uuid,schedule_uuid
0,45adf63c-48c9-3659-8561-07556d2d4ddf,344b3ebe-da7e-314c-83ed-9425269695fd,FLD,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
1,c6280e23-5e43-3859-893e-540d94993529,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,FLD,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
2,9395dc21-15d1-3fab-8d1f-6f3fe6114c48,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
3,b99e440b-39db-350a-81eb-b6eb1bd8b0bc,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
4,ca1c841f-41d5-3298-bd25-01c65751f150,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90


## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [24]:
from pyspark.sql.functions import *

df_rooms = rooms.alias('R') \
                .join(sections.alias('S'), col('R.uuid') == col('S.room_uuid'), 'left') \
                .join(course_offerings.alias('CO'), col('S.course_offering_uuid') == col('CO.uuid'), 'inner') \
                .withColumnRenamed('R.uuid', 'room_uuid') \
                .withColumnRenamed('CO.uuid', 'course_offering_uuid') \
                .withColumnRenamed('S.uuid', 'section_uuid')

df_rooms.select('room_uuid', 'facility_code', 'room_code', 'term_code', 'name').limit(5).toPandas()

,room_uuid,facility_code,room_code,term_code,name
0,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null,1172,Cooperative Education Prog
1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null,1172,Cooperative Education Prog
2,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null,1114,Cooperative Education Prog
3,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null,1104,Cooperative Education Prog
4,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null,1134,Cooperative Education Prog


In [25]:
df_rooms.printSchema()

root
 |-- uuid: string (nullable = true)
 |-- facility_code: string (nullable = true)
 |-- room_code: string (nullable = true)
 |-- uuid: string (nullable = true)
 |-- course_offering_uuid: string (nullable = true)
 |-- section_type: string (nullable = true)
 |-- number: integer (nullable = true)
 |-- room_uuid: string (nullable = true)
 |-- schedule_uuid: string (nullable = true)
 |-- uuid: string (nullable = true)
 |-- course_uuid: string (nullable = true)
 |-- term_code: integer (nullable = true)
 |-- name: string (nullable = true)



## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [13]:
df_rooms.filter(col('facility_code') == '0469') \
    .select('room_uuid', 'facility_code', 'room_code', 'term_code', 'name').limit(5).toPandas()

,room_uuid,facility_code,room_code,term_code,name
0,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,0469,2441,1152,Fundamentals-Flute
1,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,0469,2441,1092,Fundamentals-Flute
2,6af80b0b-b3e3-370a-925f-829869f007bc,0469,4411,1172,Fundamentals-Flute
3,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,0469,2441,1162,Fundamentals-Flute
4,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,0469,2441,1132,Fundamentals-Flute


## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

In [29]:
df_rooms.groupBy(['course_offering_uuid', 'facility_code']).count().limit(10).toPandas()

,course_offering_uuid,facility_code,count
0,f2df19b1-a830-33b9-8441-3b1360c1ec02,0469,1
1,2933c51b-5932-3d5f-bba7-17d401626d6f,0469,2
2,00dc2f8a-a35f-36ce-983b-d0dc14dffa36,0469,2
3,78a11bde-275c-313a-9c65-ea93faacdeba,1245,2
4,1dc33092-4519-3781-acef-c9e2c447c5b6,0054,1
5,9ca6f5ed-71e5-39c9-862c-00ba7180249c,0402,1
6,63cb23e3-ab1f-34fb-83d7-44aabcf942d3,0057,1
7,0f965313-d423-321f-a974-2d504979c999,0469,2
8,60c2273e-bc62-38d8-ab2f-01fd08dccebe,0482,9
9,34a7e6cd-cdea-38f8-8a27-e20eb41cf56a,0056,1


## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

In [49]:
df_grades = course_offerings.alias('CO') \
                .join(sections.alias('S'), col('CO.uuid') == col('S.course_offering_uuid'), 'inner') \
                .join(grade_distributions.alias('GD'), col('GD.course_offering_uuid') == col('CO.uuid'), 'inner') \
                .withColumnRenamed('CO.uuid', 'course_offering_uuid') \
                .withColumnRenamed('S.uuid', 'section_uuid')

df_grades.printSchema()

root
 |-- uuid: string (nullable = true)
 |-- course_uuid: string (nullable = true)
 |-- term_code: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- uuid: string (nullable = true)
 |-- course_offering_uuid: string (nullable = true)
 |-- section_type: string (nullable = true)
 |-- number: integer (nullable = true)
 |-- room_uuid: string (nullable = true)
 |-- schedule_uuid: string (nullable = true)
 |-- course_offering_uuid: string (nullable = true)
 |-- section_number: integer (nullable = true)
 |-- a_count: integer (nullable = true)
 |-- ab_count: integer (nullable = true)
 |-- b_count: integer (nullable = true)
 |-- bc_count: integer (nullable = true)
 |-- c_count: integer (nullable = true)
 |-- d_count: integer (nullable = true)
 |-- f_count: integer (nullable = true)
 |-- s_count: integer (nullable = true)
 |-- u_count: integer (nullable = true)
 |-- cr_count: integer (nullable = true)
 |-- n_count: integer (nullable = true)
 |-- p_count: integer (nullable = true)

In [50]:
df_grades.limit(10).toPandas()

,uuid,course_uuid,term_code,name,uuid,course_offering_uuid,section_type,number,room_uuid,schedule_uuid,...,f_count,s_count,u_count,cr_count,n_count,p_count,i_count,nw_count,nr_count,other_count
0,0005b192-c521-3b5e-9bc8-280ca9cc3eeb,2476b39f-ffc7-3d8e-b240-2edbe9b22903,1162,Independent Study,06a67510-336d-30fc-8486-fda2bc4040fa,0005b192-c521-3b5e-9bc8-280ca9cc3eeb,IND,37,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,...,0,0,0,0,0,0,0,0,0,0
1,00160703-0912-3105-88f0-2d4e26cd8dec,f527257b-ad84-38c3-9a2c-55088aee2ec5,1164,Principles of Corrosion,6df207c4-08cd-363f-8349-792f74700072,00160703-0912-3105-88f0-2d4e26cd8dec,LEC,1,d2c315a7-031f-30d4-94a9-5e18d7bc5db5,b36bbec9-5f54-3af2-9f23-e217586abbdd,...,1,0,0,0,0,0,0,0,0,0
2,00167c90-18b1-3a64-b6e5-d343f8b9d16b,d04c2f85-6a19-37f8-91a0-aa6cf659eaca,1104,Intro-Interpersonal Com,963dc529-a530-350e-af84-de54d3c14025,00167c90-18b1-3a64-b6e5-d343f8b9d16b,DIS,304,dd6118e0-7221-3b81-9b29-aad61f0ede54,c6a6dd6d-48f0-39ef-ae21-f217db9679fd,...,2,1,0,0,0,0,0,0,0,0
3,00167c90-18b1-3a64-b6e5-d343f8b9d16b,d04c2f85-6a19-37f8-91a0-aa6cf659eaca,1104,Intro-Interpersonal Com,50bf8dfe-5187-3056-a7ab-4769645eb5d8,00167c90-18b1-3a64-b6e5-d343f8b9d16b,DIS,302,4e9c55b8-7c02-36c3-b907-3e832ea285e7,a6020298-a0ee-3d53-8b0f-ff2de293e3a9,...,2,1,0,0,0,0,0,0,0,0
4,00167c90-18b1-3a64-b6e5-d343f8b9d16b,d04c2f85-6a19-37f8-91a0-aa6cf659eaca,1104,Intro-Interpersonal Com,51d4cf8e-b7d7-3deb-a0d0-caa6dab69731,00167c90-18b1-3a64-b6e5-d343f8b9d16b,DIS,301,4948c250-c6cf-3272-a497-f90962f3ba67,f9849cdf-a75d-3c78-9faa-0222a31ccf9b,...,2,1,0,0,0,0,0,0,0,0
5,00167c90-18b1-3a64-b6e5-d343f8b9d16b,d04c2f85-6a19-37f8-91a0-aa6cf659eaca,1104,Intro-Interpersonal Com,a66afad3-1488-3286-83a3-62739e988ca3,00167c90-18b1-3a64-b6e5-d343f8b9d16b,DIS,305,698555af-8ec3-3b85-8e67-bd7fc53aba80,944178ec-e75c-3e0a-85be-e489eb9748ec,...,2,1,0,0,0,0,0,0,0,0
6,00167c90-18b1-3a64-b6e5-d343f8b9d16b,d04c2f85-6a19-37f8-91a0-aa6cf659eaca,1104,Intro-Interpersonal Com,29b7eea2-8af8-3e08-bcc0-d8d3c3322d1e,00167c90-18b1-3a64-b6e5-d343f8b9d16b,LEC,1,503166b1-58ca-39e1-984f-c59bef027b11,0ebb5383-9be7-35ae-b48e-56ce64d343ab,...,2,1,0,0,0,0,0,0,0,0
7,00167c90-18b1-3a64-b6e5-d343f8b9d16b,d04c2f85-6a19-37f8-91a0-aa6cf659eaca,1104,Intro-Interpersonal Com,23a8c852-aeca-357e-8595-6e64f8e239a5,00167c90-18b1-3a64-b6e5-d343f8b9d16b,DIS,306,afb4fefd-ea1e-351e-9021-1531f9234f9a,7caed90d-786b-326d-a357-485615d8fabb,...,2,1,0,0,0,0,0,0,0,0
8,00167c90-18b1-3a64-b6e5-d343f8b9d16b,d04c2f85-6a19-37f8-91a0-aa6cf659eaca,1104,Intro-Interpersonal Com,d8a0dc1b-29be-36c6-b4c7-f21a490775d8,00167c90-18b1-3a64-b6e5-d343f8b9d16b,DIS,303,eda7ac38-2d95-3114-b675-c49966779d6d,370a1503-a6a6-3e2c-a005-fe52b8ac5558,...,2,1,0,0,0,0,0,0,0,0
9,0017d777-9164-3875-9215-caf9667002ce,569b1d63-1d1c-3504-9888-cdc85c197d71,1132,Directed Studies-Molec Biol,628aa0ac-74b3-3154-98e9-bcd5ff6aec70,0017d777-9164-3875-9215-caf9667002ce,IND,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90,...,0,0,0,0,0,0,0,0,0,0


In [61]:
df_grades_reproved = df_grades.groupBy('CO.name') \
    .agg(sum('GD.c_count').alias('c_count'), sum('GD.d_count').alias('d_count'), sum('GD.f_count').alias('f_count'))

df_grades_reproved = df_grades_reproved.withColumn('reproved', col('c_count') + col('d_count') + col('f_count'))

df_grades_reproved.select('name', 'reproved').orderBy('reproved', ascending=False).limit(1).toPandas()

,name,reproved
0,General Chemistry,1039940


## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [65]:
values = [('Monkey',10),('Monkay',36),('Mnky',123), \
          ('Elephant',48),('Elefant',16),('Ellafant',1), \
          ('Hippopotamus',48),('Hipopotamus',16),('Hippo',1)]
zoo = spark.createDataFrame(values,['Animal','age'])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



In [63]:
animals = spark.createDataFrame([('Monkey', 'Elephant', 'Hippopotamus')], ['0', '1', '2'])

In [71]:
df_animals = zoo.crossJoin(animals)
df_animals.show()

+------------+---+------+--------+------------+
|      Animal|age|     0|       1|           2|
+------------+---+------+--------+------------+
|      Monkey| 10|Monkey|Elephant|Hippopotamus|
|      Monkay| 36|Monkey|Elephant|Hippopotamus|
|        Mnky|123|Monkey|Elephant|Hippopotamus|
|    Elephant| 48|Monkey|Elephant|Hippopotamus|
|     Elefant| 16|Monkey|Elephant|Hippopotamus|
|    Ellafant|  1|Monkey|Elephant|Hippopotamus|
|Hippopotamus| 48|Monkey|Elephant|Hippopotamus|
| Hipopotamus| 16|Monkey|Elephant|Hippopotamus|
|       Hippo|  1|Monkey|Elephant|Hippopotamus|
+------------+---+------+--------+------------+



In [72]:
for i in [0, 1, 2]:
    df_animals = df_animals \
                    .withColumn('levenshtein_'+ str(i), levenshtein(df_animals['Animal'], df_animals[str(i)]))

df_animals.show()

+------------+---+------+--------+------------+-------------+-------------+-------------+
|      Animal|age|     0|       1|           2|levenshtein_0|levenshtein_1|levenshtein_2|
+------------+---+------+--------+------------+-------------+-------------+-------------+
|      Monkey| 10|Monkey|Elephant|Hippopotamus|            0|            8|           11|
|      Monkay| 36|Monkey|Elephant|Hippopotamus|            1|            7|           10|
|        Mnky|123|Monkey|Elephant|Hippopotamus|            2|            8|           12|
|    Elephant| 48|Monkey|Elephant|Hippopotamus|            8|            0|           10|
|     Elefant| 16|Monkey|Elephant|Hippopotamus|            7|            2|           11|
|    Ellafant|  1|Monkey|Elephant|Hippopotamus|            8|            3|           11|
|Hippopotamus| 48|Monkey|Elephant|Hippopotamus|           11|           10|            0|
| Hipopotamus| 16|Monkey|Elephant|Hippopotamus|           10|            9|            1|
|       Hi

In [73]:
df_animals_fixed = df_animals.withColumn('Animal', 
        when(
            (df_animals['levenshtein_0'] < df_animals['levenshtein_1']) & (df_animals['levenshtein_0'] < df_animals['levenshtein_2']), 
                animals['0']) \
        .otherwise(
            when(
                (df_animals['levenshtein_1'] < df_animals['levenshtein_0']) & (df_animals['levenshtein_1'] < df_animals['levenshtein_2']), 
                    animals['1']) \
            .otherwise(animals['2'])
        )
    )

df_animals_fixed.select('Animal', 'age').show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkey| 36|
|      Monkey|123|
|    Elephant| 48|
|    Elephant| 16|
|    Elephant|  1|
|Hippopotamus| 48|
|Hippopotamus| 16|
|      Monkey|  1|
+------------+---+



### Great job! 